In [175]:
import pandas as pd
import numpy as np
import re
from nltk.stem import WordNetLemmatizer

In [176]:
# loading the questions dataset
questions = pd.read_csv('500_questions.csv', encoding='iso-8859-1')
# loading the topics dataset
topics = pd.read_csv('topic_dataset.csv')

In [177]:
questions.head(10)

,Unnamed: 0,question
0,2254,(Avg. of 42 ) : What is a SQL view?
1,1390,Question involving Primary Key Integrity const...
2,1224,How secure are encrypted stored procedures tr...
3,1172,8. A candidate appearing for an examinatio n ...
4,1135,Consider the following areas for tuning. The o...
5,22,2. Why we cant create object for abstract class?
6,1166,6.transfering of pages from memory to peripher...
7,1693,There are 30 questions in which 2 are STAR QUE...
8,994,A) InfoPackage â ' Scheduler â ' Repair Reques...
9,374,An anchoring object is a print condition objec...


In [178]:
topics.head()

,Unnamed: 0,topic,subject
0,1115,fcfs,Operating System
1,1923,dml,Database Management System
2,835,jdbc,Object_oriented_programing
3,1866,joints,Database Management System
4,1125,buffering,Operating System


In [179]:
# creating topic related words to categorize the questions
related_words = {'fcfs':['fcfs', 'turn'], 'dml':['dml'], 'jdbc':['jdbc', 'connectivity'], 'joints':['joint', 'join', 'joined'],
                'buffering':['buffering', 'buffer', 'buffered'], 'applet':['applet'],
                'deadlock':['deadlock', 'deadlocking', 'deadlocked'], 'ddl':['ddl'],
                'polymorphism':['polymorphism', 'polymorhic'], 'caching':['caching', 'cached', 'cache'],
                'thread':['thread', 'threading', 'threaded'], 'security':['security', 'secure', 'secured'], 
                'scheduling':['scheduling', 'scheduled', 'schedule', 'scheduler'],
                'Inheritance':['inheritance', 'inherited', 'inherit', 'inherits'], 'encapsulation':['encapsulation', 'encapsule', 'encapsuled', 'encapsulate'],
                'transaction':['transaction', 'transact', 'transacted', 'transactional'], 'Object':['object'], 'sql':['sql', 'view'],
                'keys':['key']}
topic_to_subject = {'fcfs':'Operating System', 'dml':'Database Management System', 'jdbc':'Object_oriented_programing',
                 'joints':'Database Management System', 'buffering':'Operating System', 'applet':'Object_oriented_programing',
                 'deadlock':'Operating System', 'ddl':'Database Management System', 'polymorphism':'Object_oriented_programing',
                 'caching':'Operating System', 'thread':'Object_oriented_programing', 'security':'Operating System', 
                 'scheduling':'Operating System', 'Inheritance':'Object_oriented_programing', 'encapsulation':'Object_oriented_programing', 
                 'transaction':'Database Management System', 'Object':'Object_oriented_programing', 'sql':'Database Management System', 
                 'keys':'Database Management System'}

In [180]:
lemmatizer = WordNetLemmatizer()
questions['topic'] = [set() for _ in range(len(questions))]

# categorizing each question by going through each word
for i in range(len(questions)):
    for word_fromQ in re.split(r'[^\w]', questions['question'][i].lower()):
        for key, word_list in related_words.items():
            if lemmatizer.lemmatize(word_fromQ) in word_list:
                questions['topic'][i].add(key)

In [181]:
# questions having no category
print((questions['topic']==set()).sum())

25


In [182]:
# sorting the questions based on the frequency
questions.sort_values(['Unnamed: 0'], ascending=[False], axis=0, inplace=True)

In [183]:
questions.head()

,Unnamed: 0,question,topic
430,2421,A few SQL queries were also asked (find second...,{sql}
51,2420,The other 25 questions were technical question...,"{Object, sql}"
213,2418,The panel was too helpful and friendly. They t...,{sql}
63,2416,The interview was quite easy. Most of the ques...,"{joints, sql}"
127,2415,For IS/CS DBMS basic knowledge SQL queries is...,"{polymorphism, Inheritance, sql}"


In [188]:
unique_subjects = ['Operating System', 'Database Management System', 'Object_oriented_programing']

# count to get 20 questions for each subject
count = dict(zip(unique_subjects, [0 for _ in range(3)]))

# creating a new dataframe that will hold the questions topic wise
result = pd.DataFrame(index=range(60), columns = ['Subject name', 'Topic name', 'Question'])
result.index.name = 'FAQ no.'

# iterating through every row of the modified 500_questions dataframe
for row in questions.itertuples():
    for t in row.topic:
        sub = topic_to_subject[t]
        print(count)
        print(unique_subjects.index(sub))
        if count[sub] < 20:
            prop_index = unique_subjects.index(sub)*10 + count[sub]
            result['Subject name'][prop_index] = sub
            result['Topic name'][prop_index] = t
            result['Question'][prop_index] = row.question
            count[sub] += 1

{'Operating System': 0, 'Database Management System': 0, 'Object_oriented_programing': 0}
1
{'Operating System': 0, 'Database Management System': 1, 'Object_oriented_programing': 0}
2
{'Operating System': 0, 'Database Management System': 1, 'Object_oriented_programing': 1}
1
{'Operating System': 0, 'Database Management System': 2, 'Object_oriented_programing': 1}
1
{'Operating System': 0, 'Database Management System': 3, 'Object_oriented_programing': 1}
1
{'Operating System': 0, 'Database Management System': 4, 'Object_oriented_programing': 1}
1
{'Operating System': 0, 'Database Management System': 5, 'Object_oriented_programing': 1}
2
{'Operating System': 0, 'Database Management System': 5, 'Object_oriented_programing': 2}
2
{'Operating System': 0, 'Database Management System': 5, 'Object_oriented_programing': 3}
1
{'Operating System': 0, 'Database Management System': 6, 'Object_oriented_programing': 3}
1
{'Operating System': 0, 'Database Management System': 7, 'Object_oriented_progra

In [185]:
result.index = result.index + 1
# saving the file as csv
result.to_csv('result.csv')